In [1]:
!wget https://sgp.fas.org/crs/misc/IF10244.pdf

--2024-10-23 00:02:25--  https://sgp.fas.org/crs/misc/IF10244.pdf
Resolving sgp.fas.org (sgp.fas.org)... 65.9.112.71, 65.9.112.81, 65.9.112.124, ...
Connecting to sgp.fas.org (sgp.fas.org)|65.9.112.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 435826 (426K) [application/pdf]
Saving to: ‘IF10244.pdf.1’

IF10244.pdf.1       100%[===================>] 425.61K   688KB/s    in 0.6s    

2024-10-23 00:02:26 (688 KB/s) - ‘IF10244.pdf.1’ saved [435826/435826]



In [2]:
###################################
########### INGESTION #############
###################################

In [2]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from unstructured.partition.utils.constants import PartitionStrategy


loader = UnstructuredPDFLoader(
    file_path="./IF10244.pdf",
    strategy=PartitionStrategy.HI_RES,
    infer_table_structure=True,
    extract_images_in_pdf=True,
    chunking_strategy="by_title",
    new_after_n_chars=4000,  # Soft-max
    max_characters=4000,  # Hard-max
    combine_text_under_n_chars=2000,  # Combine chunks of < 200 chars
    mode='elements',  # Split the documents into elements such as Title and NarrativeText.
)
data = loader.load()

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
len(data)

7

In [4]:
data[0]

Document(page_content='= Informing the legislative debate since 1914 Congressional Research Service\n\nUpdated June 1, 2023\n\nWildfire Statistics\n\nWildfires are unplanned fires, including lightning-caused fires, unauthorized human-caused fires, and escaped fires from prescribed burn projects. States are responsible for responding to wildfires that begin on nonfederal (state, local, and private) lands, except for lands protected by federal agencies under cooperative agreements. The federal government is responsible for responding to wildfires that begin on federal lands. The Forest Service (FS)—within the U.S. Department of Agriculture—carries out wildfire management and response across the 193 million acres of the National Forest System (NFS). The Department of the Interior (DOI) manages wildfire response for more than 400 million acres of national parks, wildlife refuges and preserves, other public lands, and Indian reservations.\n\nFigure 1. Annual Wildfires and Acres Burned, 1993

In [5]:
[doc.metadata['category'] for doc in data]

['CompositeElement',
 'CompositeElement',
 'Table',
 'CompositeElement',
 'CompositeElement',
 'Table',
 'CompositeElement']

In [6]:
data[2].page_content  # Table

'2018 2019 2020 Number of Fires (thousands) Federal 12.5 10.9 14.4 FS 5.6 5.3 6.7 DOI 7.0 5.3 7.6 2021 14.0 6.2 7.6 2022 11.7 5.9 5.8 Other 0.1 0.2 <0.1 0.2 0.1 Nonfederal 45.6 39.6 44.6 45.0 57.2 Total 58.1 50.5 59.0 59.0 69.0 Acres Burned (millions) Federal 4.6 3.1 7.1 5.2 4.0 FS 2.3 0.6 4.8 4.1 1.9 DOI 2.3 2.3 2.3 1.0 2.1 Other <0.1 <0.1 <0.1 <0.1 <0.1 Nonfederal 4.1 1.6 3.1 1.9 3.6 Total 8.8 4.7 10.1 7.1 7.6'

In [7]:
###################################
########### RETRIEVER #############
###################################

In [13]:
from getpass import getpass

OPENAI_API_KEY = getpass('Enter OpenAI Key: ')

Enter OpenAI Key:  ········


In [14]:
from langchain_openai import OpenAIEmbeddings


OPENAI_EMBEDDING_MODEL = OpenAIEmbeddings(model='text-embedding-ada-002', api_key=OPENAI_API_KEY)

In [15]:
from langchain_chroma import Chroma

vectorstore = Chroma(
    collection_name='OSM-21-Oct-2024-tradition',
    embedding_function=OPENAI_EMBEDDING_MODEL,
    collection_metadata={"hnsw:space": "cosine"},
)

In [16]:
retriever = vectorstore.as_retriever()

In [17]:
from langchain.vectorstores.utils import filter_complex_metadata

# Filter complex metadata
data = filter_complex_metadata(data)
retriever.vectorstore.add_documents(data)

['a18bd38a-7097-4168-a9e7-0b294d037b53',
 '37bbd45a-6825-4de0-9156-7251e3374724',
 'a95358ec-7696-457d-acd5-b73028fcedab',
 '046c4463-ea4b-4f45-960d-2d96b59a95f6',
 '82258716-360b-4fae-a194-a90875599f7a',
 'c76d09b3-5b88-4818-b846-36f9153d5e81',
 'b3f15895-ca37-444c-8b5e-f2252dfd38bf']

In [18]:
vectorstore.get(include=["metadatas", "documents", "embeddings"])

{'ids': ['a18bd38a-7097-4168-a9e7-0b294d037b53',
  '37bbd45a-6825-4de0-9156-7251e3374724',
  'a95358ec-7696-457d-acd5-b73028fcedab',
  '046c4463-ea4b-4f45-960d-2d96b59a95f6',
  '82258716-360b-4fae-a194-a90875599f7a',
  'c76d09b3-5b88-4818-b846-36f9153d5e81',
  'b3f15895-ca37-444c-8b5e-f2252dfd38bf'],
 'embeddings': array([[-0.00673014, -0.01467681, -0.00481378, ...,  0.00020643,
         -0.0054384 , -0.04871341],
        [-0.00888838, -0.02641155,  0.00992871, ...,  0.01312775,
         -0.02176905, -0.03344683],
        [-0.01428289, -0.02292607, -0.00621024, ...,  0.00606269,
          0.00284117, -0.02788377],
        ...,
        [-0.00293701, -0.01478987, -0.00015226, ...,  0.00107496,
         -0.00784089, -0.03967039],
        [ 0.00454781, -0.01484307, -0.00538824, ..., -0.01603593,
         -0.01684925, -0.02789683],
        [-0.0055647 , -0.00070016,  0.00095794, ...,  0.00303182,
         -0.01367726, -0.03927556]]),
 'documents': ['= Informing the legislative debate since 

In [19]:
vectorstore._collection.count()

7

In [20]:
query = "Which year has the highest acres burned?"
docs = retriever.invoke(query, limit=5)

In [21]:
[doc.page_content for doc in data]

['= Informing the legislative debate since 1914 Congressional Research Service\n\nUpdated June 1, 2023\n\nWildfire Statistics\n\nWildfires are unplanned fires, including lightning-caused fires, unauthorized human-caused fires, and escaped fires from prescribed burn projects. States are responsible for responding to wildfires that begin on nonfederal (state, local, and private) lands, except for lands protected by federal agencies under cooperative agreements. The federal government is responsible for responding to wildfires that begin on federal lands. The Forest Service (FS)—within the U.S. Department of Agriculture—carries out wildfire management and response across the 193 million acres of the National Forest System (NFS). The Department of the Interior (DOI) manages wildfire response for more than 400 million acres of national parks, wildlife refuges and preserves, other public lands, and Indian reservations.\n\nFigure 1. Annual Wildfires and Acres Burned, 1993-2022\n\nFires, thous

In [22]:
###################################
########### Synthesis #############
###################################

In [23]:
from langchain_openai import ChatOpenAI

CHAT_MODEL = ChatOpenAI(model_name='gpt-4o-mini', api_key=OPENAI_API_KEY, temperature=0)

In [25]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context 
to answer the question. If you don't know the answer, just say that you don't know.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | CHAT_MODEL
    | StrOutputParser()
)

In [26]:
query = "Which year has the highest acres burned?"
response = rag_chain.invoke(query)

In [27]:
response

'The year with the highest acres burned is 2015.'

In [28]:
# Incorrect response. Correct answer - 4.1 million acres

query = query = "Tell me about the number of acres burned by wildfires for the forest service in 2021"
response = rag_chain.invoke(query)

In [29]:
response

'In 2021, the Forest Service (FS) reported that approximately 4.1 million acres were burned by wildfires.'